In [3]:
import numpy as np

file = open('rosalind.txt','r')
Lines = file.readlines()
lines = []
seq = ""


for line in Lines:
    if line.find('>') == 0:
        lines.append(seq)
        seq = ""
    if line.find('>') != 0:
            seq = seq + line.strip('\n')

lines.append(seq)
lines.pop(0)

''

In [4]:
def BLOSUM62_scoring_matrix(s1,s2):
    sMatrixTxt = '''
   A  C  D  E  F  G  H  I  K  L  M  N  P  Q  R  S  T  V  W  Y
A  4  0 -2 -1 -2  0 -2 -1 -1 -1 -1 -2 -1 -1 -1  1  0  0 -3 -2
C  0  9 -3 -4 -2 -3 -3 -1 -3 -1 -1 -3 -3 -3 -3 -1 -1 -1 -2 -2
D -2 -3  6  2 -3 -1 -1 -3 -1 -4 -3  1 -1  0 -2  0 -1 -3 -4 -3
E -1 -4  2  5 -3 -2  0 -3  1 -3 -2  0 -1  2  0  0 -1 -2 -3 -2
F -2 -2 -3 -3  6 -3 -1  0 -3  0  0 -3 -4 -3 -3 -2 -2 -1  1  3
G  0 -3 -1 -2 -3  6 -2 -4 -2 -4 -3  0 -2 -2 -2  0 -2 -3 -2 -3
H -2 -3 -1  0 -1 -2  8 -3 -1 -3 -2  1 -2  0  0 -1 -2 -3 -2  2
I -1 -1 -3 -3  0 -4 -3  4 -3  2  1 -3 -3 -3 -3 -2 -1  3 -3 -1
K -1 -3 -1  1 -3 -2 -1 -3  5 -2 -1  0 -1  1  2  0 -1 -2 -3 -2
L -1 -1 -4 -3  0 -4 -3  2 -2  4  2 -3 -3 -2 -2 -2 -1  1 -2 -1
M -1 -1 -3 -2  0 -3 -2  1 -1  2  5 -2 -2  0 -1 -1 -1  1 -1 -1
N -2 -3  1  0 -3  0  1 -3  0 -3 -2  6 -2  0  0  1  0 -3 -4 -2
P -1 -3 -1 -1 -4 -2 -2 -3 -1 -3 -2 -2  7 -1 -2 -1 -1 -2 -4 -3
Q -1 -3  0  2 -3 -2  0 -3  1 -2  0  0 -1  5  1  0 -1 -2 -2 -1
R -1 -3 -2  0 -3 -2  0 -3  2 -2 -1  0 -2  1  5 -1 -1 -3 -3 -2
S  1 -1  0  0 -2  0 -1 -2  0 -2 -1  1 -1  0 -1  4  1 -2 -3 -2
T  0 -1 -1 -1 -2 -2 -2 -1 -1 -1 -1  0 -1 -1 -1  1  5  0 -2 -2
V  0 -1 -3 -2 -1 -3 -3  3 -2  1  1 -3 -2 -2 -3 -2  0  4 -3 -1
W -3 -2 -4 -3  1 -2 -2 -3 -3 -2 -1 -4 -4 -2 -3 -3 -2 -3 11  2
Y -2 -2 -3 -2  3 -3  2 -1 -2 -1 -1 -2 -3 -1 -2 -2 -2 -1  2  7
'''

    sMatrixList = sMatrixTxt.split('\n')
    sMatrixList.pop(0)
    del sMatrixList[-1]

    col = sMatrixList[0].index(s1)
    for i in range(len(sMatrixList)):
        if sMatrixList[i][0] == s2:
            score = int(sMatrixList[i][col-1]+sMatrixList[i][col])
    
    return score

In [5]:
def local_alignmen(lines):
    v = lines[0]
    w = lines[1]
    
    n = len(v)
    m = len(w)
    
    gap_penalty = 1
    gap_opening = 11
    
    
    lower = np.ones((n+1,m+1), dtype= int)*(-1000000)
    middle = np.ones((n+1,m+1), dtype= int)*(-1000000)
    upper = np.ones((n+1,m+1), dtype= int)*(-1000000)
    local_distance = [lower, middle, upper]
    
    direction_lower = np.zeros((n+1,m+1), dtype= int)   
    direction_middel = np.zeros((n+1,m+1), dtype= int)
    direction_upper = np.zeros((n+1,m+1), dtype= int)
    direction = [direction_lower, direction_middel, direction_upper]
    
    local_distance[0][0,0] = 0
    local_distance[1][0,0] = 0
    local_distance[0][1,0] = -gap_opening
    local_distance[1][1,0] = -gap_opening

    for i in np.arange(2,n+1):
        local_distance[0][i,0] = local_distance[0][i-1,0] - gap_penalty
        local_distance[1][i,0] = local_distance[0][i,0]
    
    
    
    local_distance[2][0,0] = 0
    local_distance[2][0,1] = -gap_opening
    local_distance[1][0,1] = -gap_opening
    
    for i in np.arange(2,m+1):
        local_distance[2][0,i] = local_distance[2][0,i-1] - gap_penalty
        local_distance[1][0,i] = local_distance[2][0,i]
    
    for i in range(1,n+1):
        for j in range(1,m+1):
            check_point = [local_distance[0][i-1,j]-gap_penalty,
                          local_distance[1][i-1,j]-gap_opening]
            max_score = max(check_point)
            local_distance[0][i,j] = max_score
            if check_point.index(max_score)==1:
                direction[0][i,j] = 1 
            
            
            check_point = [local_distance[2][i,j-1]-gap_penalty,
                          local_distance[1][i,j-1]-gap_opening]
            max_score = max(check_point)
            local_distance[2][i,j] = max_score
            if check_point.index(max_score)==1:
                direction[2][i,j] = 1
            
            
            match_score = BLOSUM62_scoring_matrix(v[i-1],w[j-1])
            check_point = [(local_distance[0][i,j]),
                           (local_distance[1][i-1,j-1]+match_score),
                           (local_distance[2][i,j])]
            max_score = max(check_point)
            local_distance[1][i,j] = max_score
            if max_score == check_point[1]:
                direction[1][i,j] = 1
            elif max_score == check_point[2]:
                direction[1][i,j] = 2
    
    s1 = [] 
    s2 = []
    
    indx1 = n
    indx2 = m
    
    level = 1
    
    while indx1 >0 or indx2>0:
        
        if level == 0:
            level = direction[0][indx1,indx2]
            indx1 -= 1
            s2.append('-')
            s1.append(v[indx1])
        
        elif level == 2:
            if direction[2][indx1,indx2]==1:
                level = 1
            indx2 -= 1
            s2.append(w[indx2])
            s1.append('-')
            
        elif level == 1:
            level = direction[1][indx1,indx2]
            if level == 1:
                indx1 -= 1
                indx2 -= 1
                s1.append(v[indx1])
                s2.append(w[indx2])
                
    s1.reverse()
    s2.reverse()    
    

    return s1,s2,local_distance[1][n,m],direction,local_distance

In [6]:
s1, s2, max_score, direction, local_distance = local_alignmen(lines)

print(max_score)
x = "".join([str(i) for i in s1])
print(x)
x = "".join([str(i) for i in s2])
print(x)

226
GFTDIIWMGQCSEQFFANNNVPCARCTMVVAWPYECVGPHVQIMSDCIFHSFCYASGWMKDAQPNQTDTTGDKRKYNGIMHQEFA
GF-DIIWMGQCSEQFFANNKVPPARVLMVVAWPYECVGPHV--------HS--------KDAQMKH------KSNYNGIMHQNFA


In [12]:
s1, s2, max_score, direction, local_distance = local_alignmen(lines)

print(max_score)
x = "".join([str(i) for i in s1])
print(x)
x = "".join([str(i) for i in s2])
print(x)

226
GFTDIIWMGQCSEQFFANNNVPCARCTMVVAWPYECVGPHVQIMSDCIFHSFCYASGWMKDAQPNQTDTTGDKRKYNGIMHQEFA
GF-DIIWMGQCSEQFFANNKVPPARVLMVVAWPYECVGPHV--------HS--------KDAQMKH------KSNYNGIMHQNFA
